# Calculating Gene expression. 

This script calculates gene expression in raw read counts, RPKM and TPM (So we are looking at the raw reads and a normalizsed measure of gene expression) using matched samples of RNA-seq and ribosome profiling data. Note that in order to calculate these values properly, data outputted by the feature counts package is required. This is because Plastid appears to struggle to properly load the RNA data for Drosophilia melanogaster

### Step 1
Activate the Plastid conda environment and open up a Jupyter Lab session (alternatively, a python file can be run in this python environment)

### Step 2
Load in all of the necessary packages from Plastid

In [1]:
# Loading up the path to my tools file. 
import sys
sys.path.append('/home/keeganfl/Desktop/Work_Fall_2021/Fall_2021_Work_journal/tools/')

In [2]:
# Loading up required packages
from plastid import BAMGenomeArray, GenomeArray, VariableFivePrimeMapFactory, \
                        GTF2_TranscriptAssembler, GFF3_TranscriptAssembler, \
                        Transcript, ThreePrimeMapFactory, CenterMapFactory
import plastid
from plastid.plotting.plots import *
import numpy as np
import os
from Bio import SeqIO
import numpy
import math
import pandas as pd
from scipy import stats
import keegan_analysis_tools as kat
from statsmodels.nonparametric.smoothers_lowess import lowess
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from scipy.sparse.linalg import lsqr

In [13]:
# Define important paths. 
data_path = "/home/keeganfl/Desktop/Work_Fall_2021/genomes_&_samples/dmel/"
feat_counts_path = "/home/keeganfl/Desktop/Work_Fall_2021/genomes_&_samples/dmel/TE_results/"
save_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/TE_tables/dmel/"
p_site_path = "/home/keeganfl/Desktop/Work_Fall_2021/data_tables/p-site_offsets/dmel/"
gtf_name = "Drosophila_melanogaster.BDGP6.32.103.gtf"
mutation = 'control'
samp_num = '3'
special = "fc"

### Step 3
Load in your P-site offset files using the read_csv() function from pandas

In [4]:
# Load in the table of P-site offsets. 
p_offsets=pd.read_csv(p_site_path + mutation + '_RPF_' + samp_num + "_Aligned.toTranscriptome.out_p-site-offsets", sep="\t")

### Step 4
Load in a GTF genome annotation file into python using Plastid's GTF2_TranscriptAssembler() function. This function will load in the transcripts as an iterator of transcript type objects which we will convert to a list.

In [5]:
# Load up the genome annotation as a list of transcript objects
gtf_reads=list(GTF2_TranscriptAssembler(open(data_path + gtf_name),return_type=Transcript))

### Step 5
Load in the Bam files containing the Ribosome Profiling data and the RNA sequencing data as a Bam Genome Array using Plastid's BamGenomeArray() function.

In [6]:
# Open the read alignments, storing each dataset in a BAMGenomeArray:
my_datasets = { "ribosome_profiling" : data_path + mutation +"_RPF_" + samp_num + "_Aligned.sortedByCoord.out.bam"}
my_datasets = { K : BAMGenomeArray(V) for K,V in my_datasets.items() }

### Step 6
Map the read alignments to their corresponding P-sites. Again we are going to need to use our custom VariableThreePrimeMapFactory function in order to assign P-site offsets effectively.

In [7]:
# Map the ribosome profiling alignments to the p-site. 
my_datasets["ribosome_profiling"].set_mapping(kat.VariableThreePrimeMapFactory(p_offsets=p_offsets))

### Step 7
Create an empty dictionary in python that contains a list for each transcript in our GTF file and each metric we wish to calculate (e.g. length, number of counts, rpk).

In [8]:
# we will calculate both total counts and RPKM
metrics = ("counts","rpk","lengths")

# create an empty list for each sample and metric
my_data = { "%s_%s" % (SAMPLE,METRIC) : copy.deepcopy([])\
                                                  for SAMPLE in my_datasets.keys()\
                                                  for METRIC in metrics }

# add a list to our dictionary of lists to store transcript IDs, gene_names, and gene_IDs.
my_data["transcript_ID"] = []
my_data["gene_name"] = []
my_data["gene_ID"] = []

### Step 8
Fill up the dictionary with the required metrics and then convert the dictionary to a pandas dataframe for easier organization. 

In [9]:
for transcript in gtf_reads:
        
    # First, save the ID of the transcript we are evaluating
    if transcript.attr["transcript_biotype"] == "protein_coding":
        my_data["transcript_ID"].append(transcript.get_name())
        my_data["gene_ID"].append(transcript.attr["gene_id"])
        my_data["gene_name"].append(transcript.attr["gene_name"])

        # Iterate over each sample
        for sample_name, sample_data in my_datasets.items():
            counts = sum(transcript.get_counts(sample_data))
            length = transcript.length
            rpk   = float(counts)/(length/1000)
            my_data["%s_counts" % (sample_name)].append(counts)
            my_data["%s_rpk"   % (sample_name)].append(rpk)
            my_data["%s_lengths" % (sample_name)].append(length)
            
# Convert to a Dataframe
df = pd.DataFrame(my_data)

## Loading RNA sequencing data from feature counts

In [14]:
# Load in feature counts data
feature_counts = pd.read_csv(feat_counts_path + 'counts_' + mutation + '_RNA_' + samp_num + '.txt', delimiter = "\t", comment='#')

In [15]:
# Select our columns of interest. 
cols = ["Geneid","Length",mutation + '_RNA_' + samp_num + "_Aligned.sortedByCoord.out.bam"]
feature_counts = feature_counts[cols]

In [16]:
# Rename our columns to assist with the merge to the dataframe. 
feature_counts.rename(columns = {mutation + '_RNA_' + samp_num + "_Aligned.sortedByCoord.out.bam" : "RNA-seq_counts",
                                "Length" : "RNA-seq_lengths", "Geneid" : "gene_ID"}, inplace = True)

In [17]:
# create new columns that contain the calculations for tpm and rpkm. 
feature_counts["RNA-seq_rpk"] = feature_counts["RNA-seq_counts"]/feature_counts["RNA-seq_lengths"]
feature_counts["RNA-seq_tpm"] = feature_counts["RNA-seq_rpk"]/(sum(feature_counts["RNA-seq_rpk"])/1e6)
feature_counts["RNA-seq_rpkm"] =feature_counts["RNA-seq_counts"]/ feature_counts["RNA-seq_lengths"]*1000 * 1e6 / sum(feature_counts["RNA-seq_counts"])

In [18]:
# Merging the table from feature counts with the plastid dataframe based on the gene names. 
df_merged = pd.merge(df,feature_counts, how = "inner", on = "gene_ID")
df = df_merged

In [19]:
df

,ribosome_profiling_counts,ribosome_profiling_rpk,ribosome_profiling_lengths,transcript_ID,gene_name,gene_ID,RNA-seq_lengths,RNA-seq_counts,RNA-seq_rpk,RNA-seq_tpm,RNA-seq_rpkm
0,0.0,0.000000,264,FBtr0302344,CG40635,FBgn0085506,264,0,0.000000,0.000000,0.000000
1,0.0,0.000000,615,FBtr0300105,SteXh:CG42398,FBgn0259817,615,0,0.000000,0.000000,0.000000
2,3.0,2.100840,1428,FBtr0347010,CG41561,FBgn0085692,1428,0,0.000000,0.000000,0.000000
3,534.0,101.753049,5248,FBtr0078170,l(2)gl,FBgn0002121,5986,21635,3.614267,168.208107,84.627492
4,534.0,99.053979,5391,FBtr0078171,l(2)gl,FBgn0002121,5986,21635,3.614267,168.208107,84.627492
...,...,...,...,...,...,...,...,...,...,...,...
30705,1051.0,784.914115,1339,FBtr0433500,mt:ND4,FBgn0262952,1339,15071,11.255414,523.827421,263.543784
30706,443.0,1522.336770,291,FBtr0100880,mt:ND4L,FBgn0013683,291,148,0.508591,23.669848,11.908581
30707,469.0,893.333333,525,FBtr0100883,mt:ND6,FBgn0013685,525,99,0.188571,8.776122,4.415371
30708,3100.0,2726.473175,1137,FBtr0433502,mt:Cyt-b,FBgn0013678,1137,116568,102.522427,4771.397699,2400.546733


### Step 9
Calculate RPKM and TPM for the ribo-profile and RNA-seq data for each transcript using the equations <br />
$
RPKM = 10^9 * \frac{reads\: mapped\: to\: the\: transcript}{total\: reads * transcript\: length}
$
<br />
&
<br />
$TPM = 10^6 * \frac{reads\: mapped\: to\: the\: transcript\:/\: transcript\: length }{Sum(reads\: mapped\: to\: each\: transcript\:/\: each\: transcript's\: length)}
$
<br />
Then add them as new columns to the pandas dataframe.

In [22]:
for sample_name in ["ribosome_profiling"]:
    print(sample_name)

ribosome_profiling


In [20]:
# Calculate tpm using rpk from our new dataframe. 
for sample_name in ["ribosome_profiling"]:
    per_m_s = sum(df["%s_rpk"   % (sample_name)])/1e6
    df["%s_tpm" % (sample_name)] = (df["%s_rpk"   % (sample_name)])/per_m_s
    
# Calculate rpkm using the read counts.
for sample_name in ["ribosome_profiling"]:
    counts = df["%s_counts"   % (sample_name)]
    rpkm   = counts / df["ribosome_profiling_lengths"] * 1e9 / sum(counts)
    df["%s_rpkm" % (sample_name)] = rpkm

### Step 10
Calculate the TE by dividing the RPKM and TPM for the ribosome profiling data by the RPKM and TPM for the mRNA sequence data. Save the TE values as new columns in the pandas dataframe. 

In [ ]:
# Calculate translation efficiency using tpm
df["TE_tpm"] = df["ribosome_profiling_tpm"] / df["RNA-seq_tpm"]

# Calculate translation efficiency using rpkm
df["TE_rpkm"] = df["ribosome_profiling_rpkm"] / df["RNA-seq_rpkm"]

In [ ]:
# Rearranging Column names to make something more readable. 
cols = ['transcript_ID', "gene_ID","gene_name", 'TE_tpm','TE_rpkm','ribosome_profiling_tpm',
        'RNA-seq_tpm','ribosome_profiling_counts','RNA-seq_counts',
        'ribosome_profiling_rpkm', 'RNA-seq_rpkm', 'ribosome_profiling_rpk', 
       'RNA-seq_rpk', 'ribosome_profiling_lengths', 'RNA-seq_lengths']
df = df[cols]

### Step 11
Use a kernel density estimate plot to look at the distribution of TE's in your data. This is an important step for making sure nothing has gone wrong in the alignment, P-site offsetting, or TE calculation steps. In order to create these TE tables, we are going to need to remove all of the negative or infinite values from our data (these occur naturally as a result of there being zero reads for genes that have very low expression). Once these annoying values have been removed we can use Plastids kde_plot() function to look at the distribution of TE values. We are hoping to see the majority of our TE values being somewhere between 0 and 5 with just a small proportion of transcripts having higher TEs than that. 

In [ ]:
# We need to remove zeroes and infinite values in order to make the kernel density estimates
df_clean=df[~df.isin([np.nan, np.inf, -np.inf, 0]).any(1)]

# We are only going to look at TE values that fall within a certain region. 
df_interest=df_clean[(df_clean["TE_tpm"] >= 0)&(df_clean["TE_tpm"] <= 800)]

In [ ]:
# plot kernel density estimates of A and B on the same axis
fig, ax = kde_plot(df_interest["TE_tpm"],label="TE")

# make things look nice
plt.xlabel("TE")
plt.ylabel("Probability density")
plt.title("kde_plot() demo",y=1.1)
plt.xlim(0,20)

### Step 12
save the dataframe using df.to_csv() so it can be used in future notebooks. 

In [ ]:
# save the data frame.
df.to_csv(save_path + mutation + '_' + samp_num + '_' + "te_table" + special + ".csv", index = False)